# Colab训练版本ColorManga
    
如果您希望在Colab训练ColorManga，请使用此Notebook(切勿在本地运行)

## 特别注意
在使用Colab时您需要确保您的会话：
* 使用Python 3环境
* 使用GPU硬件 
* troch版本为1.0.1

In [ ]:
import torch
print(torch.__version__)

if not torch.cuda.is_available():
    print('无法启用GPU！')

#### 注意：如果您的troch版本不是1.0.1，则需要运行以下代码将其修改至1.0.1；执行完该代码后您需要重启代码执行程序

In [ ]:
!pip uninstall torch
!pip install torch==1.0.1 torchvision==0.2.2

### 克隆整个仓库

In [ ]:
!git clone https://github.com/jantic/DeOldify.git ColorManga 

In [ ]:
cd ColorManga

In [ ]:
#注意:  该项命令需要被优先执行
from deoldify import device
from deoldify.device_id import DeviceId
#选择您的运行设备:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

### 进行环境初始化

In [ ]:
!pip install -r requirements.txt

In [ ]:
import fastai
from deoldify.visualize import *

torch.backends.cudnn.benchmark = True

### 载入数据

In [ ]:
!pip install kaggle

__您需要登录至[Kaggle](https://www.kaggle.com/)并打开您的个人设置(Your Profile)页面，之后点开账户(Account)选项卡，在其中找到API条目。之后创建一个新的API Token并下载，将下载的json文件中的username和key填写到下方，之后执行后续代码。__

In [ ]:
import json
username = 'xxxx' #@param {type:"string"}
key = 'xxxxxx' #@param {type:"string"}
token = {"username":username,"key":key} 
with open('/content/kaggle.json', 'w') as file:
  json.dump(token, file)

In [ ]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v /content

In [ ]:
!kaggle datasets download -d mylesoneill/tagged-anime-illustrations

In [ ]:
!mv ../datasets/mylesoneill/tagged-anime-illustrations/tagged-anime-illustrations.zip ../datasets
!unzip ../datasets/tagged-anime-illustrations.zip
!rm -f ../datasets/tagged-anime-illustrations.zip

### 开始训练

In [ ]:
import os
import fastai
from fastai import *
from fastai.vision import *
from fastai.callbacks.tensorboard import *
from fastai.vision.gan import *
from deoldify.generators import *
from deoldify.critics import *
from deoldify.dataset import *
from deoldify.loss import *
from deoldify.save import *
from PIL import Image, ImageDraw, ImageFont
from PIL import ImageFile

#### 设置参数

In [ ]:
path = Path('./danbooru-images/danbooru-images/')
path_hr = path
path_lr = path/'bandw'

proj_id = 'Danbooru2017'

gen_name = proj_id + '_gen'
pre_gen_name = gen_name + '_0'
crit_name = proj_id + '_crit'

name_gen = proj_id + '_image_gen'
path_gen = path/name_gen

TENSORBOARD_PATH = Path('data/tensorboard/' + proj_id)

nf_factor = 1.5
pct_start = 1e-8

In [ ]:
def get_data(bs:int, sz:int, keep_pct:float):
    return get_colorize_data(sz=sz, bs=bs, crappy_path=path_lr, good_path=path_hr, 
                             random_seed=None, keep_pct=keep_pct)

def get_crit_data(classes, bs, sz):
    src = ImageList.from_folder(path, include=classes, recurse=True).split_by_rand_pct(0.1, seed=42)
    ll = src.label_from_folder(classes=classes)
    data = (ll.transform(get_transforms(max_zoom=2.), size=sz)
           .databunch(bs=bs).normalize(imagenet_stats))
    return data

def create_training_images(fn,i):
    dest = path_lr/fn.relative_to(path_hr)
    dest.parent.mkdir(parents=True, exist_ok=True)
    img = PIL.Image.open(fn).convert('LA').convert('RGB')
    img.save(dest)  
    
def save_preds(dl):
    i=0
    names = dl.dataset.items
    
    for b in dl:
        preds = learn_gen.pred_batch(batch=b, reconstruct=True)
        for o in preds:
            o.save(path_gen/names[i].name)
            i += 1
    
def save_gen_images():
    if path_gen.exists(): shutil.rmtree(path_gen)
    path_gen.mkdir(exist_ok=True)
    data_gen = get_data(bs=bs, sz=sz, keep_pct=0.085)
    save_preds(data_gen.fix_dl)
    PIL.Image.open(path_gen.ls()[0])

## 构造黑白训练数据

注意：该命令只需运行一次

In [ ]:
if not path_lr.exists():
    il = ImageList.from_folder(path_hr)
    parallel(create_training_images, il.items)

## GAN的预训练器

#### 注意
大部分的训练都是在GAN的训练前进行的。这里的目标是尽可能采用常规方法训练生成器，因为与GAN训练相比，常规训练更容易控制，并且不会获得错误的结果。

### 64px

In [ ]:
bs=88
sz=64
keep_pct=1.0

In [ ]:
data_gen = get_data(bs=bs, sz=sz, keep_pct=keep_pct)

In [ ]:
learn_gen = gen_learner_deep(data=data_gen, gen_loss=FeatureLoss(), nf_factor=nf_factor)

In [ ]:
learn_gen.callback_fns.append(partial(ImageGenTensorboardWriter, base_dir=TENSORBOARD_PATH, name='GenPre'))

In [ ]:
learn_gen.fit_one_cycle(1, pct_start=0.8, max_lr=slice(1e-3))

In [ ]:
learn_gen.save(pre_gen_name)

In [ ]:
learn_gen.unfreeze()

In [ ]:
learn_gen.fit_one_cycle(1, pct_start=pct_start,  max_lr=slice(3e-7, 3e-4))

In [ ]:
learn_gen.save(pre_gen_name)

### 128px

In [ ]:
bs=22
sz=128
keep_pct=1.0

In [ ]:
learn_gen.data = get_data(sz=sz, bs=bs, keep_pct=keep_pct)

In [ ]:
learn_gen.unfreeze()

In [ ]:
learn_gen.fit_one_cycle(1, pct_start=pct_start, max_lr=slice(1e-7,1e-4))

In [ ]:
learn_gen.save(pre_gen_name)

### 192px

In [ ]:
bs=11
sz=192
keep_pct=0.50

In [ ]:
learn_gen.data = get_data(sz=sz, bs=bs, keep_pct=keep_pct)

In [ ]:
learn_gen.unfreeze()

In [ ]:
learn_gen.fit_one_cycle(1, pct_start=pct_start, max_lr=slice(5e-8,5e-5))

In [ ]:
learn_gen.save(pre_gen_name)

## NoGAN训练循环

#### 注意
根据经验，最好的结果是重复约5-8次以下的循环，直到返回不再变化(图像质量没有改善)。每次重复循环时，都希望将old_checkpoint_num增加1，以便新的检查点不覆盖旧的检查点。

In [ ]:
old_checkpoint_num = 0
checkpoint_num = old_checkpoint_num + 1
gen_old_checkpoint_name = gen_name + '_' + str(old_checkpoint_num)
gen_new_checkpoint_name = gen_name + '_' + str(checkpoint_num)
crit_old_checkpoint_name = crit_name + '_' + str(old_checkpoint_num)
crit_new_checkpoint_name= crit_name + '_' + str(checkpoint_num)

### 保存生成的图像

In [ ]:
bs=8
sz=192

In [ ]:
learn_gen = gen_learner_deep(data=data_gen, gen_loss=FeatureLoss(), nf_factor=nf_factor).load(gen_old_checkpoint_name, with_opt=False)

In [ ]:
save_gen_images()

### 预训练判别器

##### 基于生成器只训练判别器，之后进行整合

In [ ]:
if old_checkpoint_num == 0:
    bs=64
    sz=128
    learn_gen=None
    gc.collect()
    data_crit = get_crit_data([name_gen, 'test'], bs=bs, sz=sz)
    data_crit.show_batch(rows=3, ds_type=DatasetType.Train, imgsize=3)
    learn_critic = colorize_crit_learner(data=data_crit, nf=256)
    learn_critic.callback_fns.append(partial(LearnerTensorboardWriter, base_dir=TENSORBOARD_PATH, name='CriticPre'))
    learn_critic.fit_one_cycle(6, 1e-3)
    learn_critic.save(crit_old_checkpoint_name)

In [ ]:
bs=16
sz=192

In [ ]:
data_crit = get_crit_data([name_gen, 'test'], bs=bs, sz=sz)

In [ ]:
data_crit.show_batch(rows=3, ds_type=DatasetType.Train, imgsize=3)

In [ ]:
learn_critic = colorize_crit_learner(data=data_crit, nf=256).load(crit_old_checkpoint_name, with_opt=False)

In [ ]:
learn_critic.callback_fns.append(partial(LearnerTensorboardWriter, base_dir=TENSORBOARD_PATH, name='CriticPre'))

In [ ]:
learn_critic.fit_one_cycle(4, 1e-4)

In [ ]:
learn_critic.save(crit_new_checkpoint_name)

### GAN训练

In [ ]:
learn_crit=None
learn_gen=None
gc.collect()

In [ ]:
lr=1e-5
sz=192
bs=9

In [ ]:
data_crit = get_crit_data([name_gen, 'test'], bs=bs, sz=sz)

In [ ]:
learn_crit = colorize_crit_learner(data=data_crit, nf=256).load(crit_new_checkpoint_name, with_opt=False)

In [ ]:
learn_gen = gen_learner_deep(data=data_gen, gen_loss=FeatureLoss(), nf_factor=nf_factor).load(gen_old_checkpoint_name, with_opt=False)

In [ ]:
switcher = partial(AdaptiveGANSwitcher, critic_thresh=0.65)
learn = GANLearner.from_learners(learn_gen, learn_crit, weights_gen=(1.0,2.0), show_img=False, switcher=switcher,
                                 opt_func=partial(optim.Adam, betas=(0.,0.9)), wd=1e-3)
learn.callback_fns.append(partial(GANDiscriminativeLR, mult_lr=5.))
learn.callback_fns.append(partial(GANTensorboardWriter, base_dir=TENSORBOARD_PATH, name='GanLearner', visual_iters=100))
learn.callback_fns.append(partial(GANSaveCallback, learn_gen=learn_gen, filename=gen_new_checkpoint_name, save_iters=100))

#### 循环进行GAN训练
进行多次epoch以找到拐点，可以通过keep_pct调整学习率，模型会保存在中./data/tensorboard/Danbooru2017中。

In [ ]:
learn.data = get_data(sz=sz, bs=bs, keep_pct=0.03)
learn_gen.freeze_to(-1)
# 控制epoch次数
learn.fit(100,lr)